In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
#================================================
# Count and print types:
#================================================

# df=pd.read_csv('..\\data\\995,000_rows.csv') 
# df.type.value_counts()

In [ ]:
#================================================
# Remove borken row
#================================================

# df=pd.read_csv('..\\data\\995,000_rows.csv')

#-----------------------------------------------
# Find broken row
#-----------------------------------------------
# row = 0
# for i in range(len(df)):
#     if df.loc[i, "type"] == '2018-02-10 13:43:39.521661' :
#         row = i
#         break
   
#-----------------------------------------------
# Save broken row
#----------------------------------------------- 
# row = 908192
# the1 = df.iloc[[row]]
# the1.to_csv('data\\the1.csv', index=False)

#-----------------------------------------------
# Save everything but broken row
#----------------------------------------------- 
# df.drop(index=row, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.to_csv('..\\data\\995,000_rows-1.csv', index=False)

In [ ]:
def XyData(csv_path, X_path, y_path):
    usecols = ['content', 'type']
    df = pd.read_csv(csv_path, usecols=usecols)
    
    df.dropna(subset=usecols, inplace=True) #remove rows with empty cells
    df.drop(df[df['type'] == 'unknown'].index, inplace=True) #remove 'unknown' label
    df.reset_index(drop=True, inplace=True) #reset index
    
    replacement = {'reliable'   : 0,
                   'political'  : 0,
                   'bias'       : 1,
                   'fake'       : 1,
                   'conspiracy' : 1,
                   'rumor'      : 1,
                   'unknown'    : 1,
                   'unreliable' : 1,
                   'clickbait'  : 1,
                   'junksci'    : 1, 
                   'satire'     : 1,
                   'hate'       : 1}
    y = pd.DataFrame(df['type'].replace(replacement))

    X = pd.DataFrame(df.content)    
         
    X.to_csv(X_path, index=False)
    y.to_csv(y_path, index=False)

# XyData('..\\news_sample.csv', 'X.csv', 'y.csv')
XyData('..\\data\\995,000_rows-1.csv', '..\\data\\X_995k.csv', '..\\data\\y_995k.csv')

In [ ]:
def Split(X_path, y_path):
    df_X = pd.read_csv(X_path)
    df_y = pd.read_csv(y_path)
    X_train, X_test, y_train, y_test = train_test_split(df_X.content,
                                                        df_y.type,
                                                        train_size = 0.8,
                                                        random_state = 5,
                                                        shuffle = True)

    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size= 0.5,
                                                    random_state = 5,
                                                    shuffle = True)

    return X_train, X_test, X_val, y_train, y_test, y_val
# X_train, X_test, X_val, y_train, y_test, y_val = Split('X.csv', 'y.csv')
X_train, X_test, X_val, y_train, y_test, y_val = Split('..\\data\\X_995k.csv', '..\\data\\y_995k.csv')

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.75)

X_train_vec = vectorizer.fit_transform(X_train)

X_test_vec = vectorizer.transform(X_test)

In [ ]:
model = PassiveAggressiveClassifier(max_iter=100)
model.fit(X_train_vec, y_train)

#model.score(X_test_vec, y_test)

y_pred = model.predict(X_test_vec)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {score}%')

In [ ]:
model = LinearSVC(max_iter=100)
model.fit(X_train_vec, y_train)

#model.score(X_test_vec, y_test)

y_pred = model.predict(X_test_vec)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {score}%')

In [ ]:
# confusion_matrix(y_test,y_pred, labels=['1','0'])

In [ ]:
# df_X = pd.read_csv(..\\data\\X_995k.csv) 
# df_y = pd.read_csv(..\\data\\X_995k.csv)

# X = vectorizer.transform(df_X.content)
# y = df_y.type.values

# scores = cross_val_score(model, X, y, cv=5)
# print(f'K Fold Accuracy: {score}%')